In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow import keras
import math
from sklearn import preprocessing
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from collections import Counter

In [101]:
import pandas as pd
import numpy as np
import random
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
import pickle
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/My Drive/IOT Application'
!ls

/content/drive/My Drive/IOT Application
'Classifier Weather.ipynb'	'Noi Bai.csv'
'Clone Weather.ipynb'		 timeseries_weather_forecasting
'Fetch API using Python.ipynb'	 Vinh.csv
 model_checkpoint_v2.h5		'Weather IoT'
 model_checkpoint_v2.json


In [106]:
df = pd.read_csv("Noi Bai.csv")
df

,Temperature,Humidity,Pressure,UV index,Conclusion
0,59.0,88.0,30.16,0,Fog
1,59.0,88.0,30.19,0,Fog
2,57.0,94.0,30.16,0,Fog
3,57.0,88.0,30.16,0,Fog
4,55.0,94.0,30.16,0,Fog
...,...,...,...,...,...
110301,91.0,63.0,29.61,5,Fair
110302,91.0,63.0,29.58,3,Partly Cloudy
110303,91.0,63.0,29.58,2,Partly Cloudy
110304,90.0,66.0,29.58,1,Mostly Cloudy


In [108]:
df['Conclusion'].value_counts()

Fair                       39921
Mostly Cloudy              31540
Fog                        14273
Partly Cloudy               9757
Light Rain                  5601
Light Drizzle               1719
Cloudy                      1636
Haze                        1492
Rain                        1100
Drizzle                      932
T-Storm                      476
Light Rain with Thunder      410
Thunder in the Vicinity      379
Smoke                        329
Thunder                      218
Heavy Rain                   198
Heavy T-Storm                192
Mostly Cloudy / Windy         33
Heavy Rain / Windy            16
Heavy T-Storm / Windy         16
Heavy Drizzle                 14
Rain Shower                   14
Rain / Windy                  13
Light Rain / Windy            11
Thunder / Windy                5
Partly Cloudy / Windy          4
T-Storm / Windy                2
Heavy Rain Shower              2
Light Rain Shower              1
Patches of Fog                 1
Fair / Win

In [109]:
label_encoder = preprocessing.LabelEncoder()

conclusion_arr = df['Conclusion'].values
label_encoder.fit(conclusion_arr)
numerical_label = label_encoder.transform(conclusion_arr)

In [110]:
df = df.drop(columns=['Conclusion'])

In [111]:
def mean(a):
  return sum(a)/len(a)

In [112]:
def fill_none(array_col):
    for i in range(len(array_col)):
      if math.isnan(array_col[i]):
        array_col[i] = mean([tmp for tmp in array_col[i-4:i+4] if math.isnan(tmp) == False])
    return array_col

In [113]:
df['Humidity']= fill_none(df['Humidity'].values)
df['Pressure'] = fill_none(df['Pressure'].values)
df['Temperature'] = fill_none(df['Temperature'].values)
df['UV index'] = fill_none(df['UV index'].values)

In [114]:
mean = df.mean(axis=0).values
std = df.std(axis=0).values
df =(df.values -mean) /std
df = pd.DataFrame(df)

In [115]:
df.columns =['Temperature', 'Humidity', 'Pressure', 'UV index']

In [116]:
df['Label'] = numerical_label

In [118]:
df['Label'].value_counts()

2     39921
17    31540
4     14273
19     9757
13     5601
12     1719
0      1636
5      1492
22     1100
1       932
26      476
16      410
30      379
25      329
28      218
7       198
10      192
18       33
11       16
8        16
6        14
24       14
23       13
14       11
29        5
20        4
9         2
27        2
21        1
3         1
15        1
Name: Label, dtype: int64

In [119]:
a = df['Label'].value_counts().index.tolist()
b = df['Label'].value_counts().tolist()

In [120]:
occur = {}
for i in range(len(a)):
  occur[a[i]] = b[i]

In [121]:
label_under_200 = []
for k, v in occur.items():
    if v < 900:
      label_under_200.append(k)

label_under_200

[26, 16, 30, 25, 28, 7, 10, 18, 11, 8, 6, 24, 23, 14, 29, 20, 9, 27, 21, 3, 15]

In [122]:
for i in label_under_200:
    df = df.drop(df[df['Label'] == i].index)

In [123]:
y = df['Label'].values

In [140]:
label_encoder_2 = preprocessing.LabelEncoder()

label_encoder_2.fit(y)
y = label_encoder_2.transform(y)

In [141]:
y

array([3, 3, 3, ..., 8, 7, 7])

In [124]:
df = df.drop(columns=['Label'])

In [125]:
X = df.values

In [143]:
X.shape

(107971, 4)

In [144]:
rus = RandomUnderSampler(random_state=0)
X_train, y_train =rus.fit_resample(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [145]:
num_folds = 10
scoring = 'accuracy'
seed = 42 

In [146]:
models = [] 
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('LinearSVC',LinearSVC()))

In [147]:
results = []
names = []
for name, model in models:
  kfold = KFold(n_splits=num_folds,random_state=seed,shuffle=True)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

LR: 0.330365 (0.014487)
KNN: 0.395815 (0.014246)
CART: 0.417060 (0.012272)
LinearSVC: 0.330579 (0.014075)
